In [1]:
from src.trainer.eval import eval
from src.model.models import get_model
from src.data.ru_go_emotions import get_dataloaders
from datasets import load_dataset
import pandas as pd
import numpy as np
from transformers import pipeline

pd.options.display.max_colwidth = 80

labels = {
    0: "admiration",
    1: "amusement",
    2: "anger",
    3: "annoyance",
    4: "approval",
    5: "caring",
    6: "confusion",
    7: "curiosity",
    8: "desire",
    9: "disappointment",
    10: "disapproval",
    11: "disgust",
    12: "embarrassment",
    13: "excitement",
    14: "fear",
    15: "gratitude",
    16: "grief",
    17: "joy",
    18: "love",
    19: "nervousness",
    20: "optimism",
    21: "pride",
    22: "realization",
    23: "relief",
    24: "remorse",
    25: "sadness",
    26: "surprise",
    27: "neutral",
}
ru_labels = [
    "восхищение",
    "веселье",
    "злость",
    "раздражение",
    "одобрение",
    "забота",
    "непонимание",
    "любопытство",
    "желание",
    "разочарование",
    "неодобрение",
    "отвращение",
    "смущение",
    "возбуждение",
    "страх",
    "признательность",
    "горе",
    "радость",
    "любовь",
    "нервозность",
    "оптимизм",
    "гордость",
    "осознание",
    "облегчение",
    "раскаяние",
    "грусть",
    "удивление",
    "нейтральность",
]
labels_id = {item:ru_labels[item] for item in labels}
labels_trans = {labels[item]:ru_labels[item] for item in labels}



In [2]:
dataset = load_dataset("seara/ru_go_emotions")
pipe = pipeline(model="seara/rubert-tiny2-ru-go-emotions")

Using custom data configuration seara--ru-go-emotions-010f1c10233a04e9
Found cached dataset parquet (/home/seara/.cache/huggingface/datasets/seara___parquet/seara--ru-go-emotions-010f1c10233a04e9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
answers = pipe(dataset["test"]["ru_text"], top_k=3)
answers

[[{'label': 'sadness', 'score': 0.6166690587997437},
  {'label': 'remorse', 'score': 0.2709709703922272},
  {'label': 'caring', 'score': 0.06799550354480743}],
 [{'label': 'disgust', 'score': 0.2813722491264343},
  {'label': 'annoyance', 'score': 0.18793629109859467},
  {'label': 'disapproval', 'score': 0.15844811499118805}],
 [{'label': 'admiration', 'score': 0.5672914385795593},
  {'label': 'joy', 'score': 0.23943160474300385},
  {'label': 'excitement', 'score': 0.23612093925476074}],
 [{'label': 'gratitude', 'score': 0.7799906730651855},
  {'label': 'confusion', 'score': 0.17765359580516815},
  {'label': 'realization', 'score': 0.16842500865459442}],
 [{'label': 'neutral', 'score': 0.7851347327232361},
  {'label': 'realization', 'score': 0.040118150413036346},
  {'label': 'excitement', 'score': 0.0255090594291687}],
 [{'label': 'gratitude', 'score': 0.7038206458091736},
  {'label': 'approval', 'score': 0.08187543600797653},
  {'label': 'confusion', 'score': 0.06059423089027405}],
 [

In [4]:
processed_answers = []
for i in range(len(answers)):
    new_array = []
    first = True
    for j in range(len(answers[i])):
        if first == True:
            new_array.append(labels_trans[answers[i][j]["label"]])
            first = False
        elif answers[i][j]["score"] >= 0.2:
            new_array.append(labels_trans[answers[i][j]["label"]])
    processed_answers.append(new_array)


In [5]:
processed_answers

[['грусть', 'раскаяние'],
 ['отвращение'],
 ['восхищение', 'радость', 'возбуждение'],
 ['признательность'],
 ['нейтральность'],
 ['признательность'],
 ['нейтральность'],
 ['признательность', 'радость'],
 ['грусть', 'раскаяние'],
 ['отвращение'],
 ['нейтральность'],
 ['любовь'],
 ['нейтральность', 'разочарование', 'грусть'],
 ['восхищение'],
 ['страх'],
 ['неодобрение', 'желание'],
 ['злость'],
 ['нейтральность', 'грусть'],
 ['нейтральность'],
 ['признательность'],
 ['раздражение', 'злость'],
 ['любопытство', 'нейтральность'],
 ['нейтральность'],
 ['нейтральность'],
 ['нейтральность'],
 ['нейтральность'],
 ['любовь'],
 ['оптимизм', 'неодобрение'],
 ['одобрение', 'восхищение'],
 ['любопытство', 'непонимание'],
 ['нейтральность'],
 ['любопытство', 'непонимание'],
 ['неодобрение', 'раздражение'],
 ['оптимизм'],
 ['нейтральность'],
 ['восхищение', 'нейтральность', 'одобрение'],
 ['нейтральность'],
 ['нейтральность'],
 ['нейтральность', 'одобрение'],
 ['восхищение'],
 ['нейтральность', 'неод

In [6]:
true_answers = dataset["test"]["labels"]

In [7]:
true_answers = [[labels_id[idsh] for idsh in item] for item in true_answers]


In [8]:
errors = pd.DataFrame({
        "text": dataset["test"]["ru_text"],
        "Предсказание": processed_answers,
        "Факт": true_answers,
    })
errors["slava"] = [len(set(a).intersection(b)) for a, b in zip(errors["Факт"], errors["Предсказание"])]

In [15]:
errors[errors["slava"] == 0].sample(5)

,text,Предсказание,Факт,slava
5320,"Мне очень жаль людей в этой ветке, которые в той или иной форме полагаются н...",[грусть],[разочарование],0
1451,Это комментарий фаната из Мэриленда. Тебе должно быть стыдно за себя.,"[злость, раздражение, отвращение]",[смущение],0
1248,"Это опасно держать вместе, иначе они могут вырваться и начать питаться други...",[нейтральность],"[неодобрение, отвращение]",0
2216,"Серьезно. Единственное, что выдавало, так это отсутствие орфографических оши...",[разочарование],"[раздражение, нейтральность]",0
2565,"Ну, вы написали этот большой комментарий, и он хорошо читается, но когда вы ...",[одобрение],"[разочарование, нейтральность]",0


In [2]:
dataset = load_dataset("seara/ru_go_emotions")
tokenizer, model = get_model("seara/rubert-tiny2-ru-go-emotions", None, None, None, "eval")
train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
    tokenizer,
    max_length=None,
    batch_size=64,
    shuffle=False,
    num_workers=4,
    pin_memory=False,
    drop_last=False,
)


Using custom data configuration seara--ru-go-emotions-010f1c10233a04e9
Found cached dataset parquet (/home/seara/.cache/huggingface/datasets/seara___parquet/seara--ru-go-emotions-010f1c10233a04e9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model


Using custom data configuration seara--ru-go-emotions-010f1c10233a04e9
Found cached dataset parquet (/home/seara/.cache/huggingface/datasets/seara___parquet/seara--ru-go-emotions-010f1c10233a04e9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/seara/.cache/huggingface/datasets/seara___parquet/seara--ru-go-emotions-010f1c10233a04e9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-36cdee0d0ffeb651.arrow
Loading cached processed dataset at /home/seara/.cache/huggingface/datasets/seara___parquet/seara--ru-go-emotions-010f1c10233a04e9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-179e0585de016483.arrow
Loading cached processed dataset at /home/seara/.cache/huggingface/datasets/seara___parquet/seara--ru-go-emotions-010f1c10233a04e9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c442862ec445e426.arrow
Loading cached processed dataset at /home/seara/.cache/huggingface/datasets/seara___parquet/seara--ru-go-emotions-010f1c10233a04e9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-11f293ab8d734717.arrow
Loading cached processed dataset at /home/seara/.cache/huggingface/datasets/

Loaded dataloaders: train 679, val 85, test 85


In [106]:
labels_trans

{'admiration': 'восхищение',
 'amusement': 'веселье',
 'anger': 'злость',
 'annoyance': 'раздражение',
 'approval': 'одобрение',
 'caring': 'забота',
 'confusion': 'непонимание',
 'curiosity': 'любопытство',
 'desire': 'желание',
 'disappointment': 'разочарование',
 'disapproval': 'неодобрение',
 'disgust': 'отвращение',
 'embarrassment': 'смущение',
 'excitement': 'возбуждение',
 'fear': 'страх',
 'gratitude': 'признательность',
 'grief': 'горе',
 'joy': 'радость',
 'love': 'любовь',
 'nervousness': 'нервозность',
 'optimism': 'оптимизм',
 'pride': 'гордость',
 'realization': 'осознание',
 'relief': 'облегчение',
 'remorse': 'раскаяние',
 'sadness': 'грусть',
 'surprise': 'удивление',
 'neutral': 'нейтральность'}

In [4]:
test_y_true, test_y_pred, df = eval(
    model, test_dataloader, labels, "multi_label_classification", True
)


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


                восхищение   веселье    злость  раздражение  одобрение  \
precision           0.6777    0.7957    0.5546       0.5556     0.5981   
recall              0.6091    0.8409    0.3333       0.0312     0.1823   
f1-score            0.6416    0.8177    0.4164       0.0592     0.2795   
support           504.0000  264.0000  198.0000     320.0000   351.0000   
auc-roc             0.9241    0.9629    0.8989       0.8069     0.7784   
wrong f1 micro      0.9368    0.9818    0.9659       0.9414     0.9392   
wrong f1 macro      0.8035    0.9040    0.6994       0.5145     0.6239   

                  забота  непонимание  любопытство  желание  разочарование  \
precision         0.5000       0.7692       0.5106   0.7143         0.0000   
recall            0.0370       0.0654       0.3380   0.1807         0.0000   
f1-score          0.0690       0.1205       0.4068   0.2885         0.0000   
support         135.0000     153.0000     284.0000  83.0000       151.0000   
auc-roc          

In [5]:
next(iter(test_dataloader))["input_ids"][0]


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([    2,    45,     1,    80,  9588, 17382,  1126,  1164,  5923,  5126,
           30,    12,  3814,    45,  3983,   531,  4834,  2066,  4177,  6527,
          539,    16, 16142, 13106,   539,    16,   537, 17162,   754,  2187,
          549,     5,     3,     0,     0,     0,     0,     0])

In [6]:
tokenizer(dataset["test"]["text"][0])


{'input_ids': [2, 45, 1, 80, 9588, 17382, 1126, 1164, 5923, 5126, 30, 12, 3814, 45, 3983, 531, 4834, 2066, 4177, 6527, 539, 16, 16142, 13106, 539, 16, 537, 17162, 754, 2187, 549, 5, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
test_y_true


array([[0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [8]:
len(test_y_true)


5427

In [9]:
len(dataset["test"]["text"])


5427

In [17]:
test_y_pred[0].argmax()

24

In [34]:
test_y_pred[0].argmax()

24

In [38]:
test_y_pred[test_y_pred >= 0.5]

array([0.5546682 , 0.58704257, 0.69311523, ..., 0.976106  , 0.58306307,
       0.9357708 ], dtype=float32)

In [24]:
np.argsort(-test_y_pred)[:,:2]

array([[24, 18],
       [ 0, 11],
       [20,  0],
       ...,
       [27,  5],
       [17,  0],
       [27,  0]])

In [29]:
errors = pd.DataFrame(
    {
        "text": dataset["test"]["ru_text"],
        "Предсказание": np.argsort(-test_y_pred)[:,:2].tolist(),
        "Факт": np.argsort(-test_y_true)[:,:2].tolist(),
    }
)
errors


,text,Предсказание,Факт
0,"Я очень сожалею о вашей ситуации :( Хотя мне нравятся имена Сапфира, Цирилла...","[24, 18]","[25, 0]"
1,"Это прекрасно, потому что это ужасно. Эт не с.","[0, 11]","[0, 25]"
2,"Фанаты Kings здесь, удачи вам, ребята! Будет интересная игра для просмотра!","[20, 0]","[13, 25]"
3,"Я этого не знала, спасибо, что научили меня кое-чему сегодня!","[15, 0]","[15, 0]"
4,"Им наскучило бродить по земле тысячи лет, и в конце концов они перешли в заг...","[27, 25]","[27, 25]"
...,...,...,...
5422,Спасибо. Мне тоже после госпитализации поставили диагноз АД 1.,"[15, 0]","[15, 0]"
5423,"Что ж, это имеет смысл.","[4, 27]","[4, 0]"
5424,Папа выдает [ИМЯ],"[27, 5]","[27, 25]"
5425,"Так рад, что обнаружил этот сабреддит пару месяцев назад. Так хорошо...","[17, 0]","[0, 25]"


In [30]:
errors["Факт"] = errors["Факт"].apply(lambda x: [labels[item] for item in x])
errors["Предсказание"] = errors["Предсказание"].apply(lambda x: [labels[item] for item in x])


In [33]:
errors

,text,Предсказание,Факт
0,"Я очень сожалею о вашей ситуации :( Хотя мне нравятся имена Сапфира, Цирилла...","[раскаяние, любовь]","[грусть, восхищение]"
1,"Это прекрасно, потому что это ужасно. Эт не с.","[восхищение, отвращение]","[восхищение, грусть]"
2,"Фанаты Kings здесь, удачи вам, ребята! Будет интересная игра для просмотра!","[оптимизм, восхищение]","[возбуждение, грусть]"
3,"Я этого не знала, спасибо, что научили меня кое-чему сегодня!","[признательность, восхищение]","[признательность, восхищение]"
4,"Им наскучило бродить по земле тысячи лет, и в конце концов они перешли в заг...","[нейтральность, грусть]","[нейтральность, грусть]"
...,...,...,...
5422,Спасибо. Мне тоже после госпитализации поставили диагноз АД 1.,"[признательность, восхищение]","[признательность, восхищение]"
5423,"Что ж, это имеет смысл.","[одобрение, нейтральность]","[одобрение, восхищение]"
5424,Папа выдает [ИМЯ],"[нейтральность, забота]","[нейтральность, грусть]"
5425,"Так рад, что обнаружил этот сабреддит пару месяцев назад. Так хорошо...","[радость, восхищение]","[восхищение, грусть]"


In [32]:
errors[~errors["Факт"].isin(errors["Предсказание"])].groupby("Факт").sample(1)


TypeError: unhashable type: 'list'